# Homework #3
Here are the imports. Please do not import anything else.

In [1]:
import graphviz

In [2]:
class FSA:
    #initialize with optional sets
    def __init__(self,start={},final={},arcs={}):
        self.start = start
        self.final = final
        self.arcs = arcs
    #get the full list of states
    def states(self):
        states = set()
        for s in self.start: states.add(s)
        for s in self.final: states.add(s)
        for a in self.arcs:
            states.add(a[0])
            states.add(a[2])
        return states
    #display the FSA
    def dot(self):
        dot = graphviz.Digraph(graph_attr={'rankdir':'LR'})
        allstates = self.states()
        nonfinal = allstates.difference(self.final)
        dot.node('s',style='invisible')
        for s in nonfinal: dot.node(str(s),shape='circle')
        for s in self.final:
            dot.node(str(s),shape='doublecircle')
        for s in self.start: dot.edge('s',str(s))
        for a in self.arcs:
            dot.edge(str(a[0]),str(a[2]),label=str(a[1]))
        return dot
    
    #acceptability form a start state
    def accept(self,s):
        for state in self.start:
            #exit as soon as one works
            zeros = []
            if self._accept(s,state,zeros):
                return True
        return False
    
    #acceptability from any state
    def _accept(self,s,st,zeros):
        '''the zeros argument is meant to be a set that
        includes all states that have been visited by
        epsilon-arcs. It starts out empty and gets
        populated as you use epsilon arcs. Once you hit
        a non-epsilon-arc, it's reset to empty, until you
        hit an epsilon arc again.
        ''' 
        #exit from the recursion
        if s == '':
            if st in self.final: return True
            else: return False
        #recursive clause
        else:
            for arc in self.arcs:
                if arc[0] == st:
                    if arc[1] == s[0]:
                        if self._accept(s[1:],arc[2],zeros):
                            return True
                    if arc[1] == '':
                        if arc[0] != arc[2]:
                            if self._accept(s,arc[2],zeros):
                                return True
        return False

    def alphabet(self):
        alphabet = set()
        for a in self.arcs:
            alphabet.add(a[1])
        return alphabet
    def isComplete(self):
        states = self.states()
        alphabet = self.alphabet()
        c = {}
        for a in self.arcs:
            if a[1] in c:
                c[a[1]].add(a[0])
            else:
                c[a[1]] = {a[0]}
        for a in c:
            if len(c[a]) < len(states):
                return False
        return True
    def complete(self):
        if self.isComplete():
            print('FSA is already complete')
        else:
            states = self.states()
            alphabet = self.alphabet()
            newstate = 0
            while newstate in states:
                newstate += 1
            c = {}
            for a in self.arcs:
                if a[1] in c:
                    c[a[1]].add(a[0])
                else:
                    c[a[1]] = {a[0]}
        start = self.start.copy()
        final = self.final.copy()
        arcs = set()
        for a in self.arcs:
            arcs.add(a)
        for symbol in alphabet:
            arcs.add((newstate,symbol,newstate))
        for symbol in c:
            if len(c[symbol]) < len(states):
                for state in states:
                    if state not in c[symbol]:
                        arcs.add((state,symbol,newstate))
        fsa = FSA(
            start=start,
            final=final,
            arcs=arcs
        )
        return fsa

In [3]:
fsa = FSA(
    start = {0},
    final = {1},
    arcs = {
        (0,'a',0),
        (0,'',1),
        (1,'b',1)
    }
)
assert fsa.accept('aabb')

In [4]:
assert not fsa.accept('ba')

In [5]:
fsa2 = FSA(
    start = {0},
    final = {1},
    arcs = {
        (0,'a',0),
        (0,'',1),
        (0,'',0),
        (1,'b',1)
    }
)
assert fsa2.accept('aabb')

In [6]:
assert fsa2.accept('bb')

In [7]:
assert not fsa2.accept('ba')

In [8]:
def newstatenames(fsa,n):
    '''generate new state names
    args:
        fsa: an existing fsa
        n: the digit to start from
    returns:
        a new fsa
    '''
    # YOUR CODE HERE
    fsa_dict ={} #this is the dictionary object to remember what states changed to what!
    
    #count number of states in the fsa
    arc_list = []
    fsa.arcs = list(fsa.arcs) #makes a list of the arcs
    for arc in fsa.arcs:
        start = arc[0]
        end = arc[2]
        if start in arc_list:
            continue
        else: 
            arc_list.append(start)
        if end in arc_list:
            continue
        else:
            arc_list.append(end)
    numberofstates = len(arc_list)
    
    #original states and arcs
    start_state = list(fsa.start)[0] #original start state
    final_state = list(fsa.final)[0] #original end state
    fsa.arcs = list(fsa.arcs) #original set of arcs
    
    #rename start and end states and save both in dictionary
    fsa_dict[start_state] = n
    fsa_dict[final_state] = n + (int(numberofstates) -1)
    
    #rename other states and save both in dictionary
    lastnamedstate = n
    for item in arc_list:
        if item in fsa_dict.keys():
            continue
        else:
            fsa_dict[item] = int(lastnamedstate) + 1
            lastnamedstate = int(lastnamedstate) + 1
    
    #rename all the arcs with the new states
    newarcs = set()
    for arc in fsa.arcs:
        start = arc[0]
        statename = arc[1]
        end = arc[2]
        newstart = 0 
        newend = 0
        if start in fsa_dict.keys():
            newstart = fsa_dict[start]
        else:
            continue
        if end in fsa_dict.keys():
            newend = fsa_dict[end]
        else:
            continue
        newarc = (newstart, statename, newend)
        newarcs.add(newarc)
    
    #Create the new FSA with the new state names and the new arcs
    newfsa = FSA(start={fsa_dict[start_state]},final={fsa_dict[final_state]},arcs=newarcs)
    
    return newfsa

In [9]:
fsa3 = FSA(
    start = {33},
    final = {10},
    arcs = {
        (33,'a',33),
        (33,'b',14),
        (14,'a',10),
        (10,'b',10)
    }
)
assert newstatenames(fsa3,4).states() == {5,6,4}

In [10]:
def concatenate(f1,f2):
    '''return the concatenation of two FSAs
    args:
        f1: the first FSA
        f2: the second FSA
    returns:
        a new FSA that's the concatenation of the two arguments
    '''
    #get start state from f1
    firstfsa = newstatenames(f1,0)
    startstate = list(firstfsa.start)[0]
    
    #get start state for f2 based on the end state for f1
    first_state_in_second_fsa = int(list(firstfsa.final)[0]) +1
    
    #get the final state from f2
    secondfsa = newstatenames(f2,first_state_in_second_fsa)
    finalstate = (list(secondfsa.final)[0])
    
    #create out new set of arcs from f1, f2, and the epsilon arc connecting the 2
    newarcs = set()
    for arc in firstfsa.arcs:
        newarcs.add(arc)
    for arc in secondfsa.arcs:
        newarcs.add(arc)
    epsilonstart = list(firstfsa.final)[0] #this is the final state of the first FSA
    epsilonarc = (epsilonstart,'',first_state_in_second_fsa)
    newarcs.add(epsilonarc)
    
    newfsa = FSA(start={startstate},final={finalstate},arcs=newarcs)
    
    return newfsa

In [11]:
fsa4 = FSA(
    start={0},
    final={1},
    arcs={
        (0,'a',0),
        (0,'b',1)
    }
)
fsa5 = FSA(
    start={0},
    final={2},
    arcs={
        (0,'c',1),
        (1,'d',2)
    }
)
fsa6 = concatenate(fsa4,fsa5)
assert fsa6.accept('aabcd')

In [12]:
fsa4 = FSA(
    start={0},
    final={1},
    arcs={
        (0,'a',0),
        (0,'b',1)
    }
)
fsa5 = FSA(
    start={0},
    final={2},
    arcs={
        (0,'c',1),
        (1,'d',2)
    }
)
fsa6 = concatenate(fsa4,fsa5)
assert fsa6.accept('abcd')

In [13]:
fsa7 = concatenate(fsa4,fsa4)
assert fsa7.accept('baaab')

In [14]:
fsa8 = concatenate(fsa5,fsa5)
assert not fsa8.accept('cd')